# 环境构建

从 TF 2.11 开始，Windows平台不再支持 CUDA 构建。如果CUDA版本过高，需要在虚拟环境中构建cuda，并将其添加到系统环境中。

**虚拟环境**  
conda create -n tf_gpu python=3.10 -y   
conda activate tf_gpu  
conda install -c conda-forge cudatoolkit=11.2 cudnn=8.1.0 -y  
python -m pip install "tensorflow<2.11"  

**系统环境**  
D:\Anaconda\envs\tensorflow\Library\bin
D:\Anaconda\envs\tensorflow\Library\lib  
D:\Anaconda\envs\tensorflow\Library\include

# 代码测试

In [1]:
import tensorflow as tf

# 检查 GPU 是否可用
gpus = tf.config.list_physical_devices('GPU')
print('GPU:', len(gpus) > 0)  # 如果有 GPU 可用，则输出 True
print('可用 GPU 设备:', gpus)

GPU: True
可用 GPU 设备: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
import tensorflow as tf

version = tf.__version__  #查看Tensorflow版本
print("tensorflow version:", version)

tensorflow version: 2.10.1


In [3]:
from tensorflow import keras

print("keras version:", keras.__version__)

keras version: 2.10.0


In [4]:
import tensorflow as tf
import timeit

with tf.device('/cpu:0'):
    cpu_a = tf.random.normal([10000, 1000])
    cpu_b = tf.random.normal([1000, 2000])
    print(cpu_a.device, cpu_b.device)

with tf.device('/gpu:0'):
    gpu_a = tf.random.normal([10000, 1000])
    gpu_b = tf.random.normal([1000, 2000])
    print(gpu_a.device, gpu_b.device)


def cpu_run():
    with tf.device('/cpu:0'):
        c = tf.matmul(cpu_a, cpu_b)
    return c


def gpu_run():
    with tf.device('/gpu:0'):
        c = tf.matmul(gpu_a, gpu_b)
    return c


# warm up	这里就当是先给gpu热热身了
cpu_time = timeit.timeit(cpu_run, number=10)
gpu_time = timeit.timeit(gpu_run, number=10)
print('warmup:', cpu_time, gpu_time)

cpu_time = timeit.timeit(cpu_run, number=10)
gpu_time = timeit.timeit(gpu_run, number=10)
print('run time:', cpu_time, gpu_time)

/job:localhost/replica:0/task:0/device:CPU:0 /job:localhost/replica:0/task:0/device:CPU:0
/job:localhost/replica:0/task:0/device:GPU:0 /job:localhost/replica:0/task:0/device:GPU:0
warmup: 0.44129290000000765 178.56908090000002
run time: 0.4177760000000035 0.0009688000000096508


In [5]:
A = tf.constant([[1, 2], [3, 4]], dtype=tf.float32)  #定义张量
x = tf.constant([[0, 10], [0, 0.5]])
b = tf.constant([[1, -1]], dtype=tf.float32)
y = tf.add(tf.matmul(A, x), b, name="result")  #y=Ax+b

In [6]:
print(y)#输出结果为tf.Tensor对象

tf.Tensor(
[[ 1. 10.]
 [ 1. 31.]], shape=(2, 2), dtype=float32)


In [7]:
print(y.numpy())#转换为numpy值

[[ 1. 10.]
 [ 1. 31.]]


In [8]:
x = tf.Variable(4.0)
y = tf.Variable(2.0)
with tf.GradientTape(persistent=True) as tape:#计算梯度
    z = x + y
    w = tf.pow(x, 2)
dz_dy = tape.gradient(z, y)
dw_dx = tape.gradient(w, x)
print(dz_dy.numpy(), dw_dx.numpy())

1.0 8.0
